# Vitessce Widget Tutorial

# Visualization of a Loom file

## 1. Import dependencies

In [ ]:
import os
from os.path import join, isfile, isdir
from urllib.request import urlretrieve
from anndata import read_loom
import numpy as np

from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    AnnDataWrapper,
)
from vitessce.data_utils import (
    optimize_adata,
    to_diamond,
    VAR_CHUNK_SIZE,
)

## 2. Download data

Download `osmFISH_SScortex_mouse_all_cells.loom` from http://loom.linnarssonlab.org/.

In [ ]:
loom_filepath = join("data", "osmFISH_SScortex_mouse_all_cells.loom")
if not isfile(loom_filepath):
    os.makedirs("data", exist_ok=True)
    urlretrieve('http://loom.linnarssonlab.org/clone/osmFISH/osmFISH_SScortex_mouse_all_cells.loom', loom_filepath)

## 3. Open Loom file with AnnData's read_loom

In [ ]:
adata = read_loom(loom_filepath, obsm_names={"tSNE": ["_tSNE_1", "_tSNE_2"], "spatial": ["X", "Y"]})

Generate pseudo-segmentations as diamond-shaped polygons centered on the spatial coordinate of each cell, and store in `adata.obsm["segmentations"]`

In [ ]:
num_cells = adata.obs.shape[0]
adata.obsm["segmentations"] = np.zeros((num_cells, 4, 2))
radius = 100
for i in range(num_cells):
    adata.obsm["segmentations"][i, :, :] = to_diamond(adata.obsm['spatial'][i, 0], adata.obsm['spatial'][i, 1], radius)

Save the AnnData object to a Zarr store:

In [ ]:
zarr_filepath = join("data", "osmFISH_SScortex_mouse_all_cells.zarr")
if not isdir(zarr_filepath) or True:
    adata = optimize_adata(
        adata,
        obs_cols=["ClusterName"],
        obsm_keys=["tSNE", "spatial", "segmentations"],
        optimize_X=True,
    )
    adata.write_zarr(zarr_filepath, chunks=[adata.shape[0], VAR_CHUNK_SIZE])

## 4. Configure Vitessce

Create a Vitessce view config.

In [ ]:
vc = VitessceConfig(schema_version="1.0.15", name='Loom Example', description='osmFISH dataset of the mouse cortex including all cells')
w = AnnDataWrapper(base_path=zarr_filepath, obs_set_elems=["obs/ClusterName"], obs_set_names=["Clusters"], obs_locations_elem="obsm/spatial", obs_segmentations_elem="obsm/segmentations", obs_embedding_elems=["obsm/tSNE"])
dataset = vc.add_dataset(name='SScortex').add_object(w)

tsne = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="tSNE")
cell_sets = vc.add_view(cm.OBS_SETS, dataset=dataset)
spatial = vc.add_view(cm.SPATIAL, dataset=dataset)

spatial_segmentation_layer_value = {
  "opacity": 1,
  "radius": 0,
  "visible": True,
  "stroked": False
}

vc.link_views([spatial], [ct.SPATIAL_ZOOM, ct.SPATIAL_TARGET_X, ct.SPATIAL_TARGET_Y, ct.SPATIAL_SEGMENTATION_LAYER], [-6.43, 10417.69, 24885.55, spatial_segmentation_layer_value])
vc.layout(spatial | (tsne / cell_sets));

## 5. Render the widget

A widget can be created with the `.widget()` method on the config instance.

In [ ]:
vw = vc.widget()
vw